In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import warnings as wr

In [2]:
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import oandapyV20.endpoints.forexlabs as labs
import oandapyV20.endpoints.orders as ordersEndpoint
import oandapyV20.endpoints.trades as tradesEndpoint

### Order book data scrapped using different notebook

In [15]:
orders = pd.concat([
    pd.read_csv('../bookData/EUR_USD/orders/january.csv'),
    pd.read_csv('../bookData/EUR_USD/orders/january_missing.csv'),
    pd.read_csv('../bookData/EUR_USD/orders/february.csv'),
    pd.read_csv('../bookData/EUR_USD/orders/march.csv'),
])

In [16]:
positions = pd.concat([
    pd.read_csv('../bookData/EUR_USD/positions/january.csv'),
    pd.read_csv('../bookData/EUR_USD/positions/february.csv'),
    pd.read_csv('../bookData/EUR_USD/positions/march.csv'),
])

### Functions for getting candle data

In [17]:
def hist(api, instrument, start_days, end_days, granularity):

    start_date = (dt.datetime.now()-dt.timedelta(days=start_days)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_date = (dt.datetime.now()-dt.timedelta(days=end_days, hours=2,minutes=4)).strftime('%Y-%m-%dT%H:%M:%SZ')

    params ={
                "from": start_date,
                "to": end_date,
                "granularity":granularity,
            }

    df_list = []
    for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
        api.request(r)
        df = pd.DataFrame(r.response['candles'])
        if(df.empty==False):
            time = df['time']
            volume = pd.DataFrame(df['volume'].apply(pd.Series))
            df = pd.DataFrame(df['mid'].apply(pd.Series))
            df = pd.concat([df,time,volume], axis=1)
            df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
            #df.set_index('time',inplace=True)
            df_list.append(df)
    
    final = pd.concat(df_list)
    
    names = {
        'o': 'open',
        'c': 'close',
        'h': 'high',
        'l': 'low',
        0: 'vol',
        'time': 'time',
    }
    new_names = []
    for column_name in final.columns:
        new_names.append(names[column_name])
    final.columns = new_names
    
    return final

In [18]:
def cal(client, instrument, perdiod):

    
    
    params = {
        "instrument": instrument,
        "period": perdiod
    }
    
    # PERIOD VALUES
    #3600 - 1 hour
    #43200 - 12 hours
    #86400 - 1 day
    #604800 - 1 week
    #2592000 - 1 month
    #7776000 - 3 months
    #15552000 - 6 months
    #31536000 - 1 year
    # http://developer.oanda.com/rest-live/forex-labs/

    r = labs.Calendar(params=params)
    client.request(r)
    
    df = pd.DataFrame.from_dict(r.response, orient='columns')
    
    df['timestamp'] = pd.to_datetime(df['timestamp']*1000000000)
    df = df[['impact', 'timestamp']]
    df.columns = ['impact', 'time']

    return df.groupby('time').sum().reset_index()

In [19]:
def merge(history, calendar):
    return pd.merge(history, calendar, left_on = 'time', right_on = 'time', how='outer')\
                                                                            .set_index('time')\
                                                                            .astype(float)\
                                                                            .fillna(0)

In [20]:
def broaden_impact(df, period):
    df = df.reset_index().sort_values('time').set_index('time')

    for i in range(periods):
        df.loc[
            (df['impact'].shift(-1-i) != 0)
        ,'impact'] = df['impact'].shift(-1-i)
    
    df['impact'] = df['impact'].fillna(0)

    return df[df['low'] != 0]

## Get needed data

In [21]:
client = API(access_token='f8599fa0624567b98d6293acc87489bb-e288ec05b46b6e3d0bc753e6a2fbab48')

In [22]:
granularity_param = 'M1'

In [23]:
calendar = cal(client, 'GBP_USD', 7776000)

In [24]:
history = hist(client, 'GBP_USD', 90, 0, granularity_param)

In [25]:
merged = merge(history, calendar)

In [26]:
merged.head(1)

,open,high,low,close,vol,impact
time,,,,,,
2019-01-15 20:31:00,1.286,1.28616,1.28581,1.28582,122.0,0.0


In [27]:
orders.head(1)

,Unnamed: 0,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
0,0,2019-01-03T15:40:00Z,1.13998,1.14,0.0065,0.0978,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [28]:
positions.head(1)

,Unnamed: 0,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
0,0,2019-01-01T00:00:00Z,1.14649,1.1465,0.3569,0.1814,0.272,0.0848,0.272,0.0848,...,0.1463,0.0936,0.0,0.0,0.0,0.0,0,0,0,0


In [29]:
orders = orders.set_index(pd.to_datetime(orders['time'], format="%Y-%m-%dT%H:%M:%SZ"))

### A bit of cleaning

In [30]:
ordersNewColumns = []
for column in orders.columns:
    ordersNewColumns.append(column+'_orders')

In [31]:
orders.columns = ordersNewColumns

In [32]:
positions = positions.set_index(pd.to_datetime(positions['time'], format="%Y-%m-%dT%H:%M:%SZ"))

In [33]:
positionsNewColumns = []
for column in positions.columns:
    positionsNewColumns.append(column+'_positions')

In [34]:
positions.columns = positionsNewColumns

In [35]:
base = merged.join(orders).join(positions)\
    .drop(['Unnamed: 0_orders', 'Unnamed: 0_positions', 'price_orders',
          'roundedPrice_orders', 'price_positions', 'roundedPrice_positions',
          'time_orders', 'time_positions'], axis = 1)\
    #.dropna()\

In [36]:
base.head(2)

,open,high,low,close,vol,impact,level_0_l_orders,level_0_s_orders,level_1_up_l_orders,level_1_up_s_orders,...,level_3_down_l_positions,level_3_down_s_positions,level_4_up_l_positions,level_4_up_s_positions,level_4_down_l_positions,level_4_down_s_positions,level_5_up_l_positions,level_5_up_s_positions,level_5_down_l_positions,level_5_down_s_positions
time,,,,,,,,,,,,,,,,,,,,,
2019-01-15 20:31:00,1.2860,1.28616,1.28581,1.28582,122.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-15 20:32:00,1.2858,1.28613,1.28579,1.28580,122.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data transformation

In [37]:
# https://www.oanda.com/forex-trading/analysis/forex-order-book

In [38]:
def addSentiment(base, levelsNumber, minDifference):
    wr.filterwarnings('ignore')
    
    if levelsNumber == 0:
        base['orderBook'] = (
            base['level_0_l_orders']
        ) / (
            base['level_0_s_orders']
        )
        
        base['positionBook'] = (
            base['level_0_l_positions']
        ) / (
            base['level_0_s_positions']
        )
        
    elif levelsNumber == 1:
        base['orderBook'] = (
            base['level_0_l_orders'] +
            base['level_1_up_l_orders'] +
            base['level_1_down_l_orders']
        ) / (
            base['level_0_s_orders'] +
            base['level_1_up_s_orders'] +
            base['level_1_down_s_orders'] 
        )
        
        base['positionBook'] = (
            base['level_0_l_positions'] +
            base['level_1_up_l_positions'] +
            base['level_1_down_l_positions']
        ) / (
            base['level_0_s_positions'] +
            base['level_1_up_s_positions'] +
            base['level_1_down_s_positions']
        )
        
    elif levelsNumber == 2:
        base['orderBook'] = (
            base['level_0_l_orders'] +
            base['level_1_up_l_orders'] +
            base['level_1_down_l_orders'] +
            base['level_2_up_l_orders'] +
            base['level_2_down_l_orders']
        ) / (
            base['level_0_s_orders'] +
            base['level_1_up_s_orders'] +
            base['level_1_down_s_orders'] +
            base['level_2_up_s_orders'] +
            base['level_2_down_s_orders']
        )
        
        base['positionBook'] = (
            base['level_0_l_positions'] +
            base['level_1_up_l_positions'] +
            base['level_1_down_l_positions'] +
            base['level_2_up_l_positions'] +
            base['level_2_down_l_positions']
        ) / (
            base['level_0_s_positions'] +
            base['level_1_up_s_positions'] +
            base['level_1_down_s_positions'] +
            base['level_2_up_s_positions'] +
            base['level_2_down_s_positions']
        )
        
    elif levelsNumber == 3:
        base['orderBook'] = (
            base['level_0_l_orders'] +
            base['level_1_up_l_orders'] +
            base['level_1_down_l_orders'] +
            base['level_2_up_l_orders'] +
            base['level_2_down_l_orders'] +
            base['level_2_up_l_orders'] +
            base['level_2_down_l_orders']
        ) / (
            base['level_0_s_orders'] +
            base['level_1_up_s_orders'] +
            base['level_1_down_s_orders'] +
            base['level_2_up_s_orders'] +
            base['level_2_down_s_orders'] +
            base['level_3_up_s_orders'] +
            base['level_3_down_s_orders']
        )
        
        base['positionBook'] = (
            base['level_0_l_positions'] +
            base['level_1_up_l_positions'] +
            base['level_1_down_l_positions'] +
            base['level_2_up_l_positions'] +
            base['level_2_down_l_positions'] +
            base['level_3_up_l_positions'] +
            base['level_3_down_l_positions']
        ) / (
            base['level_0_s_positions'] +
            base['level_1_up_s_positions'] +
            base['level_1_down_s_positions'] +
            base['level_2_up_s_positions'] +
            base['level_2_down_s_positions'] +
            base['level_3_up_s_positions'] +
            base['level_3_down_s_positions']
        )
        
    base = base.fillna(value={'orderBook': 1, 'positionBook': 1})  
    base['trade'] = None
    
    base.loc[
        (base['orderBook'] > (1+minDifference) ) &
        (base['positionBook'] > (1+minDifference) )
    ,'trade'] = 'buy'
    
    base.loc[
        (base['orderBook'] < (1-minDifference) ) &
        (base['positionBook'] < (1-minDifference) )
    ,'trade'] = 'sell'
    
    wr.filterwarnings('default')
        
    return base

In [39]:
def validateWaitingConditions(candle, trade):
    if trade['type'] == 'buy':
        if candle['low'] < trade['openPrice']:
            trade['status'] = 'open'

    elif trade['type'] == 'sell':
        if candle['high'] > trade['openPrice']:
            trade['status'] = 'open'
    
    trade['waitingCandles'] = trade['waitingCandles'] - 1
    
    return trade.copy()

In [40]:
def validateCloseConditions(candle, trade):
    if trade['type'] == 'buy':
        if candle['low'] < trade['openPrice'] - trade['slPips']:
            trade['status'] = 'lost'
            trade['pips'] = trade['slPips']
        if candle['high'] > trade['openPrice'] + trade['tpPips']:
            trade['status'] = 'won'
            trade['pips'] = trade['tpPips']
    elif trade['type'] == 'sell':
        if candle['high'] > trade['openPrice'] + trade['slPips']:
            trade['status'] = 'lost'
            trade['pips'] = trade['slPips']
        if candle['low'] < trade['openPrice'] - trade['tpPips']:
            trade['status'] = 'won'
            trade['pips'] = trade['tpPips']
            
    trade['openCandles'] = trade['openCandles'] - 1
            
    return trade.copy()

In [41]:
def reset_trade(waitingCandles, openCandles, tpPips, slPips):
    return {
        'type': None,
        'status': None,
        'openPrice': None,
        'pips': None,
        'tpPips': tpPips,
        'slPips': slPips,
        'waitingCandles': waitingCandles,
        'openCandles': openCandles
    }.copy()

In [42]:
def addImpact(candles, openCandles):
    candles['broadenImpact'] = candles['impact'].rolling(5).sum()
    candles['broadenImpact'] = candles['impact'].rolling(openCandles+5).sum().shift(openCandles+4)
    return candles

In [43]:
def processCandles(candles,openingHour,endingHour,openCandles,levelsNumber,minDifference):
    
    candles = addSentiment(candles, levelsNumber, minDifference)
    candles = addImpact(candles, openCandles)
    candles = candles\
                [['open', 'high', 'low', 'close', 'broadenImpact', 'trade']]\
                .reset_index().to_dict('rows')
    
    return candles

In [44]:
def trade(candles,candleMinutes,minMovement,tpPips,slPips,waitingCandles):
    trade = reset_trade(waitingCandles, openCandles, tpPips, slPips)
    positions = []
                                        
    for candle in candles:
        if trade['status'] == 'waiting':
            if trade['waitingCandles'] != 0:
                trade = validateWaitingConditions(candle, trade)
                trade['time'] = candle['time']
                if trade['status'] == 'open':
                    trade = validateCloseConditions(candle, trade)
                    if trade['status'] in ['lost','won']:
                        positions.append(trade)
                        trade = reset_trade(waitingCandles, openCandles, tpPips, slPips)
            else:
                trade = reset_trade(waitingCandles, openCandles, tpPips, slPips)
        elif trade['status'] == 'open':
            if trade['openCandles'] != 0:
                trade = validateCloseConditions(candle, trade)
                if trade['status'] in ['lost','won']:
                    positions.append(trade)
                    trade = reset_trade(waitingCandles, openCandles, tpPips, slPips)
                else:
                    if trade['type'] == 'buy':
                        if candle['close'] < trade['openPrice'] - (spread * 2):
                            trade['slPips'] = trade['slPips'] - 0.0001
                    else:
                        if candle['close'] > trade['openPrice'] + (spread * 2):
                            trade['slPips'] = trade['slPips'] - 0.0001
            else:
                if trade['type'] == 'buy':
                    trade['pips'] = candle['open'] - trade['openPrice']
                else:
                    trade['pips'] = trade['openPrice'] - candle['open']

                trade['status'] = 'expired'
                positions.append(trade)
                trade = reset_trade(waitingCandles, openCandles, tpPips, slPips)
        else:
            if candle['time'].minute in candleMinutes and candle['broadenImpact'] == 0:
                if candle['trade'] == 'buy':
                    trade['status'] = 'waiting'
                    trade['type'] = 'buy'
                    trade['openPrice'] = candle['open'] -\
                                        minMovement

                elif candle['trade'] == 'sell':
                    trade['status'] = 'waiting'
                    trade['type'] = 'sell'
                    trade['openPrice'] = candle['open'] +\
                                        minMovement

                if trade['status'] == 'waiting':
                    trade = validateWaitingConditions(candle, trade)

                if trade['status'] == 'open':
                    trade = validateCloseConditions(candle, trade)
                    if trade['status'] in ['lost','won']:
                        positions.append(trade)
                        trade = reset_trade(waitingCandles, openCandles, tpPips, slPips)
    return positions

In [46]:
df_list = []
spread = 0.0002

for openingHour in [8,9,10]:
    for endingHour in [15,16,17]:
        for openCandles in [5,10,15,20]:
            for levelsNumber in [1,2]:
                for minDifference in [0.1, 0.2]:
                    
                    candles = base[
                        (base.index.hour >= openingHour) &
                        (base.index.hour <= endingHour +round(openCandles/60,0) + 1)
                    ]
                    
                    candles = processCandles(
                        candles,
                        openingHour,
                        endingHour,
                        openCandles,
                        levelsNumber,
                        minDifference        
                    )

                    for candleMinutes in [[0,20,40]]:
                        for minMovement in [0.0003, 0.0005]:
                            for tpPips in [0.0010, 0.0015, 0.0020]:
                                for slPips in [0.0010, 0.0015]:
                                    for waitingCandles in [1,3,5]:
                                        
                                        positions = trade(
                                            candles,
                                            candleMinutes,
                                            minMovement,
                                            tpPips,
                                            slPips,
                                            waitingCandles
                                        )
                                        
                                        if len(positions) != 0:

                                            finalPips = 0
                                            finalWons = 0
                                            finalExpired = 0
                                            finalLosts = 0
                                            allTrades = 0
                                            allLongs = 0
                                            allShorts = 0

                                            PlusTrades = 0

                                            for position in positions:
                                                if position['status'] == 'lost':
                                                    finalLosts += 1
                                                if position['status'] == 'won':
                                                    finalWons += 1
                                                if position['status'] == 'expired':
                                                    finalExpired += 1

                                                if position['type'] == 'buy':
                                                    allLongs += 1
                                                if position['type'] == 'sell':
                                                    allShorts += 1

                                                allTrades += 1
                                                finalPips += position['pips'] - spread

                                                if position['pips'] > (0.0002 + spread):
                                                    PlusTrades += 1

                                            row = []
                                            row.append(finalPips)
                                            row.append(PlusTrades/len(positions))
                                            row.append(finalWons)
                                            row.append(finalLosts)
                                            row.append(finalExpired)
                                            row.append(allTrades)
                                            row.append(allLongs)
                                            row.append(allShorts)

                                            row.append(openingHour)
                                            row.append(endingHour)
                                            row.append(openCandles)
                                            row.append(levelsNumber)
                                            row.append(minDifference)
                                            row.append(candleMinutes)
                                            row.append(minMovement)
                                            row.append(tpPips)
                                            row.append(slPips)
                                            row.append(waitingCandles)

                                            df_list.append(row)

In [47]:
scores = pd.DataFrame(df_list, columns = [
    'Pips',
    'Acc',
    'Wons',
    'Losts',
    'Expired',
    'Trades',
    'Longs',
    'Shorts',
    
    'openHour',
    'endHour',
    'openCandles',
    'lvlNumber',
    'minDiff',
    'candleMinutes',
    'minMovement',
    'tp',
    'sl',
    'waitingCandles'
])

In [48]:
# 0.82 bez trailing stop loss

In [49]:
len(scores[scores['Pips'] > 0]) / len(scores)

0.8601466049382716

In [50]:
scores\
    [scores['Trades'] > 20]\
    [scores['Wons'] > 3]\
    .sort_values([
        'Pips'
    ],
    ascending = [
        False
    ])\
    .head(20)

/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Pips,Acc,Wons,Losts,Expired,Trades,Longs,Shorts,openHour,endHour,openCandles,lvlNumber,minDiff,candleMinutes,minMovement,tp,sl,waitingCandles
440,0.05765,0.831579,14,52,29,95,66,29,8,15,20,1,0.1,"[0, 20, 40]",0.0003,0.0015,0.0010,5
446,0.05763,0.829787,6,51,37,94,66,28,8,15,20,1,0.1,"[0, 20, 40]",0.0003,0.0020,0.0010,5
1016,0.05754,0.800000,16,58,36,110,79,31,8,16,20,1,0.1,"[0, 20, 40]",0.0003,0.0015,0.0010,5
1022,0.05719,0.798165,7,57,45,109,79,30,8,16,20,1,0.1,"[0, 20, 40]",0.0003,0.0020,0.0010,5
1598,0.05513,0.758621,7,57,52,116,86,30,8,17,20,1,0.1,"[0, 20, 40]",0.0003,0.0020,0.0010,5
1010,0.05491,0.836364,33,54,23,110,79,31,8,16,20,1,0.1,"[0, 20, 40]",0.0003,0.0010,0.0010,5
1592,0.05465,0.760684,15,58,44,117,86,31,8,17,20,1,0.1,"[0, 20, 40]",0.0003,0.0015,0.0010,5
1586,0.05252,0.794872,32,54,31,117,86,31,8,17,20,1,0.1,"[0, 20, 40]",0.0003,0.0010,0.0010,5
434,0.05219,0.863158,29,48,18,95,66,29,8,15,20,1,0.1,"[0, 20, 40]",0.0003,0.0010,0.0010,5
449,0.05203,0.688889,8,31,51,90,63,27,8,15,20,1,0.1,"[0, 20, 40]",0.0003,0.0020,0.0015,5


In [51]:
scores.to_csv('./book/eurusd_with_trailing_sl.csv')

FileNotFoundError: [Errno 2] No such file or directory: './book/eurusd_with_trailing_sl.csv'